In [41]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# TODO: Change the Income Scope

In [42]:
'''
Single family detached, 1940-1979, all income 
Multifamily with 5+ units, 1940-1979, all income 
Single family detached, 1980+, all income 
Multifamily with 5+ units, 1980+, all income 
Single family detached, 1940-1979, 0-80% AMI 
Multifamily with 5+ units, 1940-1979, 0-80% AMI 
Single family detached, 1980+, 0-80% AMI 
Multifamily with 5+ units, 1980+, 0-80% AMI 
Only interested in electric and natural gas fuel types 
'''

#'0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%'
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

# 0. Add Segment Label

In [ ]:
community = 'columbia'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")

Define segement

In [44]:
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['1940s', '1950s', '1960s', '1970s']
seg1_tenure = ['Renter', 'Owner']
seg1_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']

seg2_building_type = ['Multi-Family with 5+ Units']
seg2_vintage = ['1940s', '1950s', '1960s', '1970s']
seg2_tenure = ['Renter', 'Owner']
seg2_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1980s', '1990s', '2000s', '2010s']
seg3_tenure = ['Renter','Owner']
seg3_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1980s', '1990s', '2000s', '2010s']
seg4_tenure = ['Renter','Owner']
seg4_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']

label each building sample

In [45]:
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg1_tenure) &  
                     community_results['build_existing_model.geometry_wall_type'].isin(seg1_construction) & 
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg2_tenure) &  
                     community_results['build_existing_model.geometry_wall_type'].isin(seg2_construction) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg3_tenure) &   
                     community_results['build_existing_model.geometry_wall_type'].isin(seg3_construction) &
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg4_tenure) &   
                     community_results['build_existing_model.geometry_wall_type'].isin(seg4_construction) &
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'

# 1. Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [46]:
def figure_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    print(results['upgrade_cost_usd'])

figure_mean_upgrade_cost(community_results) 

upgrade_cost_usd per dwelling unit
upgrade_name
Electric Cooking                                                    1115.8
Electric Clothes Dryer                                              1253.0
Induction Cooking                                                   1918.7
Heat Pump Clothes Dryer                                             2521.5
Heat Pump Water Heater                                              3128.7
Basic Enclosure                                                     4004.6
Enhanced Enclosure                                                  5940.4
Mininum Efficiency Whole Home Electrification                      13639.7
Mininum Efficiency Heat Pump with Existing Heat Backup             14026.2
Mininum Efficiency Heat Pump with Electric Heat Backup             14128.6
High Efficiency Heat Pump with Electric Heat Backup                26553.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        30593.6
Enhanced Enclosure + HPWH + High Efficiency HP/Elect

## 1.2 Total Annual Savings

In [47]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>mmbtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')

figure_total_savings(community_results)

In [48]:
# flavor 2 - site, electricity, and non-electricity
def figure_total_savings(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>mmbtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_2.svg')

figure_total_savings(community_results)

In [49]:
# flavor 3 - site energy only
def figure_total_savings(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

figure_total_savings(community_results) 

In [50]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    print(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')

figure_mean_savings(community_results) 

sample_weight
upgrade_name
Electric Clothes Dryer                                              4096.305418
Electric Cooking                                                   18234.147833
Induction Cooking                                                  92049.895058
Heat Pump Clothes Dryer                                            78779.108592
Heat Pump Water Heater                                             92144.655665
Mininum Efficiency Whole Home Electrification                      38136.075473
Mininum Efficiency Heat Pump with Existing Heat Backup             92755.987940
Mininum Efficiency Heat Pump with Electric Heat Backup             93536.674242
Basic Enclosure                                                    91629.445286
Enhanced Enclosure                                                 92408.930375
High Efficiency Heat Pump with Electric Heat Backup                93536.674242
High Efficiency Whole Home Electrification                         93698.000000
Basic Enclosu

# 2. Results for Segments

## 2.1 Upgrade Cost Per Dwelling Unit 

In [51]:
def figure_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3 = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4 = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd']/segment3['sample_weight']
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd']/segment4['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    segment3 = segment3.sort_values('upgrade_cost_usd')
    segment4 = segment4.sort_values('upgrade_cost_usd')
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd'].round(1)
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    print(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    print(segment2['upgrade_cost_usd'])
    print('segment3: upgrade cost (usd) per dwelling unit')
    print(segment3['upgrade_cost_usd'])
    print('segment4: upgrade cost (usd) per dwelling unit')
    print(segment4['upgrade_cost_usd'])

figure_mean_upgrade_cost(community_results) 

segment1: upgrade cost (usd) per dwelling unit
upgrade_name
Electric Cooking                                                    1115.8
Electric Clothes Dryer                                              1253.0
Induction Cooking                                                   1918.7
Heat Pump Clothes Dryer                                             2521.5
Heat Pump Water Heater                                              3133.1
Basic Enclosure                                                     7018.2
Enhanced Enclosure                                                  9756.2
Mininum Efficiency Whole Home Electrification                      16209.2
Mininum Efficiency Heat Pump with Existing Heat Backup             17741.1
Mininum Efficiency Heat Pump with Electric Heat Backup             17961.2
High Efficiency Heat Pump with Electric Heat Backup                29715.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        34855.8
Enhanced Enclosure + HPWH + High Efficie

 ## 2.2 Annual Savings Per Dollar Invested

In [52]:
def figure_most_impact_package_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['upgrade_cost_usd']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    segment3_agg[indicator] = segment3_agg[indicator].round(5)
    segment4_agg[indicator] = segment4_agg[indicator].round(5)
    
    print('segment1')
    print(segment1_agg[indicator])
    print('segment2')
    print(segment2_agg[indicator])
    print('segment3')
    print(segment3_agg[indicator])
    print('segment4')
    print(segment4_agg[indicator])    

 ### Annual Bill Savings Per Dollar Invested

In [53]:
indicator ='saving_cbill.total_usd'
figure_most_impact_package_savings_per_dollar(indicator, community_results) 

segment1
upgrade_name
Basic Enclosure                                                    0.08762
Enhanced Enclosure                                                 0.07173
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04099
Heat Pump Water Heater                                             0.04006
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03956
Basic Enclosure + High Efficiency Whole Home Electrification       0.03746
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03634
High Efficiency Heat Pump with Electric Heat Backup                0.03531
High Efficiency Whole Home Electrification                         0.03371
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02534
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02527
Mininum Efficiency Whole Home Electrification                      0.01531
Heat Pump Clothes Dryer                                            0.01478
Ind

 ### Annual Energy Savings Per Dollar Invested

In [54]:
indicator = 'saving_energy.total_mmbtu'
figure_most_impact_package_savings_per_dollar(indicator, community_results) 

segment1
upgrade_name
Basic Enclosure                                                    0.00331
Enhanced Enclosure                                                 0.00272
Mininum Efficiency Whole Home Electrification                      0.00241
Heat Pump Water Heater                                             0.00220
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00166
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00159
Electric Cooking                                                   0.00154
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00153
Basic Enclosure + High Efficiency Whole Home Electrification       0.00151
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00146
High Efficiency Whole Home Electrification                         0.00145
High Efficiency Heat Pump with Electric Heat Backup                0.00144
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00115
Hea

 ### Annual Emission Savings Per Dollar Invested

In [55]:
indicator ='saving_emission.total_kgCO2e'
figure_most_impact_package_savings_per_dollar(indicator, community_results) 

segment1
upgrade_name
Basic Enclosure                                                    0.24065
Enhanced Enclosure                                                 0.19904
Heat Pump Water Heater                                             0.15034
Mininum Efficiency Whole Home Electrification                      0.14423
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.11972
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.11525
Basic Enclosure + High Efficiency Whole Home Electrification       0.10924
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.10571
High Efficiency Heat Pump with Electric Heat Backup                0.10408
High Efficiency Whole Home Electrification                         0.10323
Mininum Efficiency Heat Pump with Electric Heat Backup             0.10302
Electric Cooking                                                   0.09892
Mininum Efficiency Heat Pump with Existing Heat Backup             0.08384
Hea

## 2.3 Annual Total Savings

In [56]:
def figure_most_impact_package_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)

    print('segment1')
    print(segment1_agg[indicator])
    print('segment2')
    print(segment2_agg[indicator])
    print('segment3')
    print(segment3_agg[indicator])
    print('segment4')
    print(segment4_agg[indicator])    

### Annual Total Bill Savings

In [57]:
indicator ='saving_cbill.total_usd'
figure_most_impact_package_total_savings(indicator, community_results)

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    26188783.5
Basic Enclosure + High Efficiency Whole Home Electrification       25789865.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     25480462.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        25081543.7
High Efficiency Whole Home Electrification                         22015669.4
High Efficiency Heat Pump with Electric Heat Backup                18418011.6
Enhanced Enclosure                                                 12284982.8
Basic Enclosure                                                    10795361.9
Mininum Efficiency Heat Pump with Electric Heat Backup              7968124.7
Mininum Efficiency Heat Pump with Existing Heat Backup              7892844.5
Mininum Efficiency Whole Home Electrification                       2836823.8
Heat Pump Water Heater                                              2197122.7
Heat Pump Clothes Dryer                   

### Annual Total Energy Savings

In [58]:
indicator ='saving_energy.total_mmbtu'
figure_most_impact_package_total_savings(indicator, community_results)

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1052226.1
Basic Enclosure + High Efficiency Whole Home Electrification       1041875.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1023331.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1012980.6
High Efficiency Whole Home Electrification                          943942.5
High Efficiency Heat Pump with Electric Heat Backup                 753078.8
Mininum Efficiency Heat Pump with Electric Heat Backup              481931.4
Enhanced Enclosure                                                  466031.5
Mininum Efficiency Whole Home Electrification                       446293.2
Basic Enclosure                                                     408266.9
Mininum Efficiency Heat Pump with Existing Heat Backup              357123.5
Heat Pump Water Heater                                              120829.5
Heat Pump Clothes Dryer                               

### Annual Total Emission Savings

In [59]:
indicator ='saving_emission.total_kgCO2e'
figure_most_impact_package_total_savings(indicator, community_results)

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    76173212.7
Basic Enclosure + High Efficiency Whole Home Electrification       75203493.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     74227341.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        73257621.9
High Efficiency Whole Home Electrification                         67413737.9
High Efficiency Heat Pump with Electric Heat Backup                54291767.1
Enhanced Enclosure                                                 34090010.1
Mininum Efficiency Heat Pump with Electric Heat Backup             32484213.7
Basic Enclosure                                                    29648605.7
Mininum Efficiency Whole Home Electrification                      26729391.6
Mininum Efficiency Heat Pump with Existing Heat Backup             26112702.6
Heat Pump Water Heater                                              8244572.4
Heat Pump Clothes Dryer                   

## 2.4 Annual Savings Per Dwelling Unit

In [60]:
def figure_most_impact_package_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)
    
    print('segment1')
    print(segment1_agg[indicator])
    print('segment2')
    print(segment2_agg[indicator])
    print('segment3')
    print(segment3_agg[indicator])
    print('segment4')
    print(segment4_agg[indicator])

### Annual Bill Savings Per Dwelling Unit

In [61]:
indicator ='saving_cbill.total_usd'
figure_most_impact_package_mean_savings(indicator, community_results) 

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1491.8
Basic Enclosure + High Efficiency Whole Home Electrification       1469.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1451.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1428.7
High Efficiency Whole Home Electrification                         1254.1
High Efficiency Heat Pump with Electric Heat Backup                1049.2
Enhanced Enclosure                                                  699.8
Basic Enclosure                                                     614.9
Mininum Efficiency Heat Pump with Electric Heat Backup              453.9
Mininum Efficiency Heat Pump with Existing Heat Backup              449.6
Mininum Efficiency Whole Home Electrification                       248.1
Heat Pump Water Heater                                              125.5
Heat Pump Clothes Dryer                                              37.3
Induction Cookin

### Annual Energy Savings Per Dwelling Unit

In [62]:
indicator ='saving_energy.total_mmbtu'
figure_most_impact_package_mean_savings(indicator, community_results) 

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    59.9
Basic Enclosure + High Efficiency Whole Home Electrification       59.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     58.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        57.7
High Efficiency Whole Home Electrification                         53.8
High Efficiency Heat Pump with Electric Heat Backup                42.9
Mininum Efficiency Whole Home Electrification                      39.0
Mininum Efficiency Heat Pump with Electric Heat Backup             27.5
Enhanced Enclosure                                                 26.5
Basic Enclosure                                                    23.3
Mininum Efficiency Heat Pump with Existing Heat Backup             20.3
Heat Pump Water Heater                                              6.9
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                   

### Annual Percentage Energy Savings Per Dwelling Unit

In [63]:
indicator ='pct_saving_energy.total_%'
figure_most_impact_package_mean_savings(indicator, community_results) 

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    62.2
Basic Enclosure + High Efficiency Whole Home Electrification       61.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     60.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        59.8
High Efficiency Whole Home Electrification                         55.5
High Efficiency Heat Pump with Electric Heat Backup                44.1
Mininum Efficiency Whole Home Electrification                      35.0
Enhanced Enclosure                                                 27.3
Mininum Efficiency Heat Pump with Electric Heat Backup             27.0
Basic Enclosure                                                    23.8
Mininum Efficiency Heat Pump with Existing Heat Backup             20.8
Heat Pump Water Heater                                              7.4
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                   

### Annual Emission Savings Per Dwelling Unit

In [64]:
indicator ='saving_emission.total_kgCO2e'
figure_most_impact_package_mean_savings(indicator, community_results) 

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4339.1
Basic Enclosure + High Efficiency Whole Home Electrification       4283.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4228.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4173.0
High Efficiency Whole Home Electrification                         3840.1
High Efficiency Heat Pump with Electric Heat Backup                3092.6
Mininum Efficiency Whole Home Electrification                      2337.8
Enhanced Enclosure                                                 1941.9
Mininum Efficiency Heat Pump with Electric Heat Backup             1850.4
Basic Enclosure                                                    1688.9
Mininum Efficiency Heat Pump with Existing Heat Backup             1487.5
Heat Pump Water Heater                                              471.0
Electric Cooking                                                    110.4
Heat Pump Clothe

## 2.5. Pecent Stock with Net Positive Bill Savings

In [65]:
def figure_positive_negative_savings(segment, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}.svg')   

### Segment 1

In [66]:
segment = 'segment1'
figure_positive_negative_savings(segment, community_results) 

### Segment 2

In [67]:
segment = 'segment2'
figure_positive_negative_savings(segment, community_results) 

### Segment 3

In [68]:
segment = 'segment3'
figure_positive_negative_savings(segment, community_results) 

### Segment 4

In [69]:
segment = 'segment4'
figure_positive_negative_savings(segment, community_results) 

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [70]:
def figure_most_impact_package_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results.to_csv('test.csv')
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    segment3_agg['baseline_energy_burden_2023_cbills.%'] = segment3_agg['baseline_energy_burden_2023_cbills.%']/segment3_agg['sample_weight']
    segment4_agg['baseline_energy_burden_2023_cbills.%'] = segment4_agg['baseline_energy_burden_2023_cbills.%']/segment4_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    segment3_agg['energy_burden_diff'] = segment3_agg['baseline_energy_burden_2023_cbills.%'] - segment3_agg[indicator]
    segment4_agg['energy_burden_diff'] = segment4_agg['baseline_energy_burden_2023_cbills.%'] - segment4_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment3_agg = segment3_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment4_agg = segment4_agg.sort_values(['energy_burden_diff'], ascending=False)
    
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
    segment3_agg = segment3_agg[segment3_agg.upgrade_name != 'baseline']
    segment4_agg = segment4_agg[segment4_agg.upgrade_name != 'baseline']
    
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment3_agg = segment3_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment4_agg = segment4_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment3_agg['before'] = segment3_agg['before'].round(1)
    segment4_agg['before'] = segment4_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    segment3_agg['after'] = segment3_agg['after'].round(1)
    segment4_agg['after'] = segment4_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    print(segment1_agg[['before', 'after']])
    print('segment2')
    print(segment2_agg[['before', 'after']])
    print('segment3')
    print(segment3_agg[['before', 'after']])
    print('segment4')
    print(segment4_agg[['before', 'after']])

figure_most_impact_package_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 

energy burden (%) before and after upgrade
segment1
                                                                 before  after
upgrade_name                                                                  
Enhanced Enclosure + High Efficiency Whole Home Electrification    14.9    6.9
Basic Enclosure + High Efficiency Whole Home Electrification       14.9    7.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     14.9    7.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        14.9    7.2
High Efficiency Whole Home Electrification                         14.9    8.1
High Efficiency Heat Pump with Electric Heat Backup                14.9    9.1
Enhanced Enclosure                                                 14.9   11.1
Basic Enclosure                                                    14.9   11.6
Mininum Efficiency Heat Pump with Electric Heat Backup             14.9   12.2
Mininum Efficiency Heat Pump with Existing Heat Backup             14.9   12.3


C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_19820\733330893.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_19820\733330893.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_19820\733330893.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\ipyk